In [1]:
import pandas as pd
import numpy as np
import spacy
import re
from bs4 import BeautifulSoup

In [2]:
def extract_headers(soup):
  headers = []
  for header_tag in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
    for header in soup.find_all(header_tag):
      headers.append(header.get_text(strip=True))
  headers = ' '.join(headers)
  return headers

def extract_paragraphs(soup):
  contents = []
  for paragraph in soup.find_all('p'):
    contents.append(paragraph.get_text(strip=True))
  contents = ' '.join(contents)
  return contents
  

In [3]:
df_clean = pd.read_csv('../data/claims_clean.csv')
df_raw = pd.read_csv('../data/claims_raw.csv')

df_raw['soup'] = df_raw['text_tmp'].apply(lambda x: BeautifulSoup(x, 'html.parser'))
df_raw['headers'] = df_raw['soup'].apply(extract_headers)
df_raw['paragraphs'] = df_raw['soup'].apply(extract_paragraphs)

df_temp = pd.concat((df_clean['text_clean'], df_raw[['headers', 'paragraphs']]), axis=1) # concat h, p, and clean texts
df_temp = df_temp.apply(lambda row: '\n'.join(row.values.astype(str)), axis=1) # join them to a single cell
df_model = pd.concat((df_temp, df_raw[['mclass', 'bclass']]), axis = 1) # add y to the df

df_model.rename(columns={0: 'contents'}, inplace=True)

In [ ]:
df_model.to_csv('../data/data_1_after_html_extraction.csv', index=False)